# Data Section

### The data which will be used for the project is from the page https://en.wikipedia.org/wiki/Postcodes_in_Australia
### It contains two tables;
    1. A table that contains the state/territories along with their abbreviations
    2. A table containing the postcodes along with locality and state/territory abbreviations
Both tables will be joined using the common column i.e. abbreviations so that it will be easier for the user to understand which state/territory the locality belongs to.

### This section will contain the following parts;
    1. Extracting the tables from the wikipedia page
    2. Cleaning of extracted data
    3. Converting the data into dataframes
    4. Combining dataframes to get a final dataframe
    5. Using the geocoders package to get the latitude and longitude for each locality
    6. Adding the coordinates to the dataframe

#### First 3 steps will be combined together

### 1 2 and 3. Extracting, Cleaning and Converting data from Wikipedia page

In [2]:
#getting the wikipedia page
import requests
url = requests.get('https://en.wikipedia.org/wiki/Postcodes_in_Australia').text

#Using beautiful soup to convert url to xml
from bs4 import BeautifulSoup
soup = BeautifulSoup(url,"lxml")

In [3]:
#exract the table from the wikipedia page
table = soup.findAll('table',{'class':'wikitable'})

In [4]:
#Table one: contains abbreviations along with their expanded forms
print(table[0])

<table class="wikitable">
<tbody><tr>
<th>State/Territory</th>
<th>Abbreviation</th>
<th>Postcode range
</th></tr>
<tr>
<td><a href="/wiki/New_South_Wales" title="New South Wales">New South Wales</a></td>
<td>NSW</td>
<td>1000—1999 <i>(LVRs and PO Boxes only)</i><br/>2000—2599<br/>2619—2899<br/>2921—2999
</td></tr>
<tr>
<td><a href="/wiki/Australian_Capital_Territory" title="Australian Capital Territory">Australian Capital Territory</a>
</td>
<td>ACT</td>
<td>0200—0299 <i>(LVRs and PO Boxes only)</i><br/>2600—2618<br/>2900—2920
</td></tr>
<tr>
<td><a href="/wiki/Victoria_(Australia)" title="Victoria (Australia)">Victoria</a>
</td>
<td>VIC</td>
<td>3000—3999<br/>8000—8999 <i>(LVRs and PO Boxes only)</i>
</td></tr>
<tr>
<td><a href="/wiki/Queensland" title="Queensland">Queensland</a></td>
<td>QLD</td>
<td>4000—4999<br/>9000—9999 <i>(LVRs and PO Boxes only)</i>
</td></tr>
<tr>
<td><a href="/wiki/South_Australia" title="South Australia">South Australia</a></td>
<td>SA</td>
<td>5000—5799<br

In [5]:
#extract the contents of the table one
table1 = table[0].find_all('td')

In [6]:
# creating empty dataframe
import pandas as pd
column_name = ['State/Territory','Abbreviation']
df = pd.DataFrame(columns=column_name)
df

,State/Territory,Abbreviation


In [7]:
#populating the first dataframe
import re
i = 0
while i < len(table1):
    st = re.findall(r'<td>(.*?)</td>',str(table1[i]))
    if len(st) == 0:
        st = re.findall(r'<td>(.*?\n)</td>',str(table1[i])) 
    if '<a href'in st[0]:
        st1 = re.findall(r'>(.*?)</a>',st[0])
        if len(st1) == 0:
            st1 = re.findall(r'>(.*?\n)</a>',st[0])
        st2 = st1[0]
    else:
        st2 = st[0]
    
    abb = re.findall(r'<td>(.*?)</td>',str(table1[i+1]))
    a = abb[0]
    
    row = [st2,a]
    df.loc[len(df)] = row
    i+=3

In [8]:
#Checking the dataframe
df

,State/Territory,Abbreviation
0,New South Wales,NSW
1,Australian Capital Territory,ACT
2,Victoria,VIC
3,Queensland,QLD
4,South Australia,SA
5,Western Australia,WA
6,Tasmania,TAS
7,Northern Territory,NT


In [9]:
#Table two: contains postcode, locality and abbreviations
print(table[1])

<table class="wikitable">
<tbody><tr>
<th>Postcode</th>
<th>Locality</th>
<th>State derived from<br/>Postcode ranges</th>
<th>Actual State<br/>for this locality
</th></tr>
<tr>
<td>4825</td>
<td>ALPURRURULAM</td>
<td>QLD</td>
<td>NT
</td></tr>
<tr>
<td>0872</td>
<td>ERNABELLA</td>
<td>NT</td>
<td>SA
</td></tr>
<tr>
<td>0872</td>
<td>FREGON</td>
<td>NT</td>
<td>SA
</td></tr>
<tr>
<td>0872</td>
<td>INDULKANA</td>
<td>NT</td>
<td>SA
</td></tr>
<tr>
<td>0872</td>
<td>MIMILI</td>
<td>NT</td>
<td>SA
</td></tr>
<tr>
<td>0872</td>
<td>NGAANYATJARRA-GILES</td>
<td>NT</td>
<td>WA
</td></tr>
<tr>
<td>0872</td>
<td>GIBSON DESERT NORTH</td>
<td>NT</td>
<td>WA
</td></tr>
<tr>
<td>0872</td>
<td>GIBSON DESERT SOUTH</td>
<td>NT</td>
<td>WA
</td></tr>
<tr>
<td>2406</td>
<td>MUNGINDI</td>
<td>NSW</td>
<td>QLD
</td></tr>
<tr>
<td>2540</td>
<td>HMAS CRESWELL</td>
<td>NSW</td>
<td><a href="/wiki/Jervis_Bay_Territory" title="Jervis Bay Territory">Jervis Bay Territory</a>
</td></tr>
<tr>
<td>2540</td>
<td>JER

In [10]:
#extract the contents of the table two
table2 = table[1].find_all('td')

In [11]:
# creating empty dataframe
column_names = ['PostCode','Locality','Abbreviation']
df1 = pd.DataFrame(columns=column_names)
df1

,PostCode,Locality,Abbreviation


In [12]:
#populating the second dataframe
i = 0
while i < len(table2):
    postcode = re.findall(r'<td>(.*?)</td>',str(table2[i]))
    pc = postcode[0]
    locality = re.findall(r'<td>(.*?)</td>',str(table2[i+1]))
    l = locality[0]
    abbrev = re.findall(r'<td>(.*?)</td>',str(table2[i+2]))
    abb = abbrev[0]
    
    row = [pc,l,abb]
    df1.loc[len(df1)] = row
    i+=4

In [13]:
#Checking the dataframe
df1

,PostCode,Locality,Abbreviation
0,4825,ALPURRURULAM,QLD
1,0872,ERNABELLA,NT
2,0872,FREGON,NT
3,0872,INDULKANA,NT
4,0872,MIMILI,NT
5,0872,NGAANYATJARRA-GILES,NT
6,0872,GIBSON DESERT NORTH,NT
7,0872,GIBSON DESERT SOUTH,NT
8,2406,MUNGINDI,NSW
9,2540,HMAS CRESWELL,NSW


#### Note: for the second table we have ignored the fourth column which also contains abbreviations to avoid any confusion and also because the rows containing 'Jervis Bay Territory' will get lost when the dataframes are merged as it is not present in the first dataframe and we do not want to lose any data. 
#### So we have decided to go for the 3rd column only, so that all the rows are present and no data is lost

### 4. Combining dataframes into a final dataframe

In [15]:
final_df = pd.merge(df,df1,on="Abbreviation")
del final_df['Abbreviation']
final_df.head()

,State/Territory,PostCode,Locality
0,New South Wales,2406,MUNGINDI
1,New South Wales,2540,HMAS CRESWELL
2,New South Wales,2540,JERVIS BAY
3,New South Wales,2620,HUME
4,New South Wales,2620,KOWEN FOREST


In [16]:
# Grouping the Neighborhoods according to Postal Code
final_df = final_df.groupby(['PostCode','State/Territory'])['Locality'].apply(', '.join).reset_index()
final_df

,PostCode,State/Territory,Locality
0,0872,Northern Territory,"ERNABELLA, FREGON, INDULKANA, MIMILI, NGAANYAT..."
1,2406,New South Wales,MUNGINDI
2,2540,New South Wales,"HMAS CRESWELL, JERVIS BAY"
3,2611,Australian Capital Territory,"COOLEMAN, BIMBERI, BRINDABELLA, URIARRA"
4,2620,New South Wales,"HUME, KOWEN FOREST, OAKS ESTATE, THARWA, TOP NAAS"
5,3500,Victoria,PARINGI
6,3585,Victoria,MURRAY DOWNS
7,3586,Victoria,MALLAN
8,3644,Victoria,"BAROOGA, LALALTY"
9,3691,Victoria,LAKE HUME VILLAGE


### Part 5 Using geocoders package to get latitude and longtitude of each postcode

In [17]:
#Importing necessary packages
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
import geopy.geocoders 

print('Library imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/ibm/conda/miniconda3

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ca-certificates-2018.8.24  |       ha4d7672_0         136 KB  conda-forge
    conda-4.5.11               |           py35_0         636 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    openssl-1.0.2p             |       h470a237_0         3.5 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0             conda-forge
    geopy:       

In [18]:
#Calculating latitude and longitude of each row
from geopy.geocoders import Nominatim
geolocator = Nominatim()
place = 'Australia'
latitude = []
longitude = []
for index, rows in final_df.iterrows():
    pc = str(rows['PostCode'])
    addr = pc+','+place
    location = geolocator.geocode(addr)
    lat = location.latitude
    long = location.longitude
    latitude.append(lat)
    longitude.append(long)

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


### 6. Adding coordinates to the dataframe

In [19]:
#assigning the latitude and longitude to the dataframe
final_df['Latitude'] = latitude
final_df['Longitude'] = longitude

In [20]:
#Final dataframe
final_df

,PostCode,State/Territory,Locality,Latitude,Longitude
0,0872,Northern Territory,"ERNABELLA, FREGON, INDULKANA, MIMILI, NGAANYAT...",-25.719898,131.957835
1,2406,New South Wales,MUNGINDI,-29.030752,149.188191
2,2540,New South Wales,"HMAS CRESWELL, JERVIS BAY",-34.983059,150.603134
3,2611,Australian Capital Territory,"COOLEMAN, BIMBERI, BRINDABELLA, URIARRA",-35.316175,149.010503
4,2620,New South Wales,"HUME, KOWEN FOREST, OAKS ESTATE, THARWA, TOP NAAS",-35.277574,149.236242
5,3500,Victoria,PARINGI,-34.196090,142.142540
6,3585,Victoria,MURRAY DOWNS,-35.342298,143.558281
7,3586,Victoria,MALLAN,-35.402222,143.654334
8,3644,Victoria,"BAROOGA, LALALTY",-35.911559,145.671953
9,3691,Victoria,LAKE HUME VILLAGE,-36.162044,146.961706


#### This will be the final dataset that we will use to analyse the neighborhoods